In [10]:
import pandas as pd

# ### per persoon per training de gemiddelde 500_split berekenen

# def gemid_split(data):
#     data['year'] = pd.to_datetime(data['2k datum']).dt.year
    
#     data['unique_naam'] = data['naam'] + '_' + data['year'].astype(str)

#     gemid_data = data #(data.groupby(['unique_naam', 'binary_trainingtype'], as_index=False).agg({'ervaring': 'first', 'geslacht': 'first', 'gewichtsklasse': 'first', '500_split': 'mean', '2k tijd': 'first', 'binary_geslacht': 'first', 'binary_gewichtsklasse': 'first'}))

#     return gemid_data

# final_gemid_df = gemid_split(almost_clean_df)
# print(final_gemid_df.head())
# final_gemid_df.to_csv('final_gemid_df.csv', index=False)

data = pd.read_csv('final_gemid_df.csv')

In [11]:
### een dictionary maken van alle gegevens per persoon; dit is meer voor ons overzicht en niet nodig voor het model

import json

def dict_per_naam(data):
  naam_dict = {}

  for i in range(len(data)):
    row = data.iloc[i].to_dict()

    for key, value in row.items():
        if isinstance(value, pd.Timestamp):
            row[key] = value.strftime('%Y-%m-%d')

    name = row.pop('unique_naam')
    constants = {key: row.pop(key) for key in ['ervaring', 'binary_geslacht', 'binary_gewichtsklasse', '2k tijd']}

    if name not in naam_dict:
        naam_dict[name] = {'gegevens': constants, 'tijden': []}

    tijd_data = {key: row[key] for key in ['binary_trainingtype', '500_split']}
    naam_dict[name]['tijden'].append(tijd_data)

  print(json.dumps(naam_dict, indent=4, sort_keys=False))

  return naam_dict

data_dict = dict_per_naam(data)
print(len(data_dict))

{
    "z47_2018": {
        "gegevens": {
            "ervaring": 1,
            "binary_geslacht": 0,
            "binary_gewichtsklasse": 1,
            "2k tijd": 379.9
        },
        "tijden": [
            {
                "binary_trainingtype": 0,
                "500_split": 104.6
            },
            {
                "binary_trainingtype": 0,
                "500_split": 104.7
            },
            {
                "binary_trainingtype": 0,
                "500_split": 104.3
            },
            {
                "binary_trainingtype": 0,
                "500_split": 104.0
            },
            {
                "binary_trainingtype": 0,
                "500_split": 104.1
            },
            {
                "binary_trainingtype": 10,
                "500_split": 115.5
            },
            {
                "binary_trainingtype": 10,
                "500_split": 117.0
            },
            {
                "binary_trainingtype": 

In [13]:
import pandas as pd
import numpy as np

# Flatten dictionary while preserving binary_trainingtype
def preprocess_data_with_trainingtype(data_dict):
    rows = []
    
    for name, person_data in data_dict.items():
        gegevens = person_data['gegevens']
        tijden = person_data['tijden']
        
        # Create a dictionary to store average splits per binary_trainingtype
        split_per_type = {}
        for training in tijden:
            training_type = training['binary_trainingtype']
            if training_type not in split_per_type:
                split_per_type[training_type] = []
            split_per_type[training_type].append(training['500_split'])
        
        # Aggregate splits for each binary_trainingtype
        split_features = {
            f'split_{training_type}': np.mean(splits)
            for training_type, splits in split_per_type.items()
        }
        
        # Combine constants with split features
        row = {
            'unique_naam': name,
            'ervaring': gegevens['ervaring'],
            'binary_geslacht': gegevens['binary_geslacht'],
            'binary_gewichtsklasse': gegevens['binary_gewichtsklasse'],
            '2k_tijd': gegevens['2k tijd'],
            **split_features  # Add split features
        }
        rows.append(row)
    
    # Fill missing training types with 0 (no data for that type)
    return pd.DataFrame(rows).fillna(0)

# Example usage
df = preprocess_data_with_trainingtype(data_dict)
print(df)


    unique_naam  ervaring  binary_geslacht  binary_gewichtsklasse  2k_tijd  \
0      z47_2018         1                0                      1    379.9   
1      z50_2018         1                0                      1    390.2   
2      z52_2018         1                0                      1    382.2   
3      z54_2018         1                0                      1    392.0   
4       z4_2021         1                0                      1    381.6   
..          ...       ...              ...                    ...      ...   
167   D255_2024         0                1                      1    471.7   
168   D254_2024         0                1                      1    469.8   
169  Z2522_2024         1                0                      1    382.8   
170  Z2517_2024         1                0                      1    399.1   
171  Z2519_2024         1                0                      1    377.0   

     split_0    split_10     split_1    split_11  split_100  ..

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 1: Load the data
# Assuming your dataset is already in a DataFrame called `data`
# If it's stored in a file:
# data = pd.read_csv("your_data.csv")

# Step 2: Define features (X) and target (y)
X = data.drop(columns=['unique_naam', '2k_tijd'])  # Drop target and non-numeric identifiers
y = data['2k_tijd']

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 5: Make predictions
y_pred = model.predict(X_test)

# Step 6: Evaluate the model
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))


Mean Squared Error: 566.5932527988025
Mean Absolute Error: 12.883422196771509
R-squared: 0.5103538243039465
